In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [3]:
import re
results_page = BeautifulSoup(response.content,'lxml')

#### get data from January 2009 through June 2015

In [13]:
def get_taxi_link():
    link_array = []
    pattern = r"(http.*yellow_tripdata_(20[0|1][0-9])-(0?\d*).parquet)"
    # pattern = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet"
    match = re.findall(pattern, str(results_page))
    for i in match:
        if int(i[1]) < 2015:
            link_array.append(i[0])
        elif int(i[1]) == 2015 and int(i[2]) <= 6:
            link_array.append(i[0])
    return link_array

In [14]:
link = get_taxi_link()
link

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-06.pa

In [6]:
pattern = r"yellow.*.parquet"
match = re.search(pattern, link[0])
match.group()

'yellow_tripdata_2019-01.parquet'

In [45]:
import os

if os.path.exists("yellow_tripdata_2009-01.parquet"):
    print("true")
else:
    print("doesn't exist")

true


download data to local folder

In [25]:
import os

pattern = r"yellow.*.parquet"
for taxi_data_url in link:
    match = re.search(pattern, taxi_data_url)
    name_of_file = match.group()
    
    # to make sure we're not re-downloading the same file again
    if not os.path.exists(name_of_file):
        response = requests.get(taxi_data_url, stream=True)
        with open(name_of_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                

In [8]:
link[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet'

#### get all column names to normalize columns

In [ ]:
import os
import re

import pandas as pd
from pyarrow.parquet import ParquetFile

pattern = r"yellow.*.parquet"
parquet_column_list = []

for file in os.listdir():
    match = re.match(pattern, file)
    if match:
        file_name = match.group()
        df = pd.read_parquet(file_name)
        column_name = df.columns.to_list()
        if column_name not in parquet_column_list:
            parquet_column_list.append(column_name)

In [4]:
parquet_column_list

[['VendorID',
  'tpep_pickup_datetime',
  'tpep_dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'RatecodeID',
  'store_and_fwd_flag',
  'PULocationID',
  'DOLocationID',
  'payment_type',
  'fare_amount',
  'extra',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'improvement_surcharge',
  'total_amount',
  'congestion_surcharge',
  'airport_fee'],
 ['vendor_name',
  'Trip_Pickup_DateTime',
  'Trip_Dropoff_DateTime',
  'Passenger_Count',
  'Trip_Distance',
  'Start_Lon',
  'Start_Lat',
  'Rate_Code',
  'store_and_forward',
  'End_Lon',
  'End_Lat',
  'Payment_Type',
  'Fare_Amt',
  'surcharge',
  'mta_tax',
  'Tip_Amt',
  'Tolls_Amt',
  'Total_Amt'],
 ['vendor_id',
  'pickup_datetime',
  'dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'pickup_longitude',
  'pickup_latitude',
  'rate_code',
  'store_and_fwd_flag',
  'dropoff_longitude',
  'dropoff_latitude',
  'payment_type',
  'fare_amount',
  'surcharge',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'to

In [5]:
import pandas as pd

In [6]:
uber_data = pd.read_csv('uber_rides_sample.csv')
uber_data

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1


In [9]:
# change the format of key to datetime
uber_data['key'] = pd.to_datetime(uber_data['key'])

examine how many rows are there for each year and month

In [13]:
examine_df = pd.concat([uber_data['key'].dt.year, uber_data['key'].dt.month], axis=1)
examine_df.columns = ['year', 'month']

In [17]:
examine_df.groupby(['year', 'month']).size()

year  month
2009  1        2555
      2        2292
      3        2703
      4        2574
      5        2660
               ... 
2015  2        2225
      3        2362
      4        2339
      5        2449
      6        2161
Length: 78, dtype: int64

In [18]:
# number of rows needed for each month
200000/78

2564.102564102564

#### according to the average number of rows in each month each year in the sample uber dataset, we do the following:
- sample 2600 columns from the yellow taxi dataset
- remove unecessary column and normalize the yellow taxi columns by mapping their names to match the uber dataset

In [36]:
def get_center_long(LocationID):
    return 0
def get_center_lat(LocationID):
    return 0

In [42]:
yellow_taxi_data = pd.DataFrame()
pattern = r"yellow.*.parquet"

count = 0
for file in os.listdir():
    match = re.match(pattern, file)
    if match:
        file_name = match.group()
        df = pd.read_parquet(file_name)
        sample_df = df.sample(2600)
        column_name = sample_df.columns.to_list()
        
        if parquet_column_list.index(column_name) == 0:
            sample_df_column_filtered = sample_df.loc[:,['fare_amount',
                                                         'tpep_pickup_datetime',
                                                          'passenger_count']]
            #'PULocationID', 'DOLocationID' will be transformed to longitude and latitude
            sample_df_column_filtered['pickup_longitude'] = sample_df_column_filtered.apply(get_center_long)
            sample_df_column_filtered['pickup_latitude'] = sample_df_column_filtered.apply(get_center_lat)
            sample_df_column_filtered['dropoff_longitude'] = sample_df_column_filtered.apply(get_center_long)
            sample_df_column_filtered['dropoff_latitude'] = sample_df_column_filtered.apply(get_center_lat)
            sample_df_column_filtered.columns = ['fare_amount', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
        elif parquet_column_list.index(column_name) == 1:
            sample_df_column_filtered = sample_df.loc[:,[ 'Trip_Pickup_DateTime',            
                                                          'Passenger_Count',            
                                                          'Start_Lon',
                                                          'Start_Lat',              
                                                          'End_Lon',
                                                          'End_Lat',
                                                          'Fare_Amt']]
            sample_df_column_filtered.columns = ['pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount']
        elif parquet_column_list.index(column_name) == 2:
            sample_df_column_filtered = sample_df.loc[:,[ 'pickup_datetime',  
                                                          'passenger_count',
                                                          'pickup_longitude',
                                                          'pickup_latitude',
                                                          'dropoff_longitude',
                                                          'dropoff_latitude',  
                                                          'fare_amount']]
        yellow_taxi_data = pd.concat([yellow_taxi_data, sample_df_column_filtered])
        count += 1

In [43]:
yellow_taxi_data

,fare_amount,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
8954006,3.3,2011-07-20 12:20:00,2,NaN,NaN,NaN,NaN
8937481,5.7,2011-07-20 11:26:00,1,NaN,NaN,NaN,NaN
7466248,13.7,2011-07-17 07:13:00,5,NaN,NaN,NaN,NaN
8633905,16.5,2011-07-19 19:08:11,1,NaN,NaN,NaN,NaN
1726253,24.9,2011-07-05 15:36:18,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6238420,4.5,2009-05-25 12:39:50,3,-73.982551,40.758776,-74.001096,40.762793
3202282,6.5,2009-05-26 03:00:00,5,-73.992338,40.758823,-74.004393,40.733435
7795767,8.5,2009-05-26 19:58:01,1,-73.970145,40.759599,-73.987793,40.746540
4622530,3.7,2009-05-16 12:46:00,2,-73.950548,40.775438,-73.944485,40.783447


In [3]:
column_name = df.columns


In [12]:
large_list = [column_name.to_list(), column_name.to_list()]
if column_name.to_list() in large_list:
    print("success")
else:
    print("fail")

success


In [13]:
empty_list = []
empty_list.append(large_list)

In [14]:
large_list

[['VendorID',
  'tpep_pickup_datetime',
  'tpep_dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'RatecodeID',
  'store_and_fwd_flag',
  'PULocationID',
  'DOLocationID',
  'payment_type',
  'fare_amount',
  'extra',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'improvement_surcharge',
  'total_amount',
  'congestion_surcharge',
  'airport_fee'],
 ['VendorID',
  'tpep_pickup_datetime',
  'tpep_dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'RatecodeID',
  'store_and_fwd_flag',
  'PULocationID',
  'DOLocationID',
  'payment_type',
  'fare_amount',
  'extra',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'improvement_surcharge',
  'total_amount',
  'congestion_surcharge',
  'airport_fee']]

In [15]:
empty_list

[[['VendorID',
   'tpep_pickup_datetime',
   'tpep_dropoff_datetime',
   'passenger_count',
   'trip_distance',
   'RatecodeID',
   'store_and_fwd_flag',
   'PULocationID',
   'DOLocationID',
   'payment_type',
   'fare_amount',
   'extra',
   'mta_tax',
   'tip_amount',
   'tolls_amount',
   'improvement_surcharge',
   'total_amount',
   'congestion_surcharge',
   'airport_fee'],
  ['VendorID',
   'tpep_pickup_datetime',
   'tpep_dropoff_datetime',
   'passenger_count',
   'trip_distance',
   'RatecodeID',
   'store_and_fwd_flag',
   'PULocationID',
   'DOLocationID',
   'payment_type',
   'fare_amount',
   'extra',
   'mta_tax',
   'tip_amount',
   'tolls_amount',
   'improvement_surcharge',
   'total_amount',
   'congestion_surcharge',
   'airport_fee']]]

In [6]:
import numpy as np
#array = np.array()
np.concatenate((np.array(column_name), np.array(column_name)))

array(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID',
       'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee', 'VendorID',
       'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'airport_fee'], dtype=object)

In [ ]:
filename = "yellow_tripdata_2009-01.parquet"
df = pd.read_parquet(link[0])

In [10]:
index = df.columns

In [11]:
np.array(index)

array(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID',
       'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee'], dtype=object)

In [23]:
import geopandas as gpd
shapefile = gpd.read_file("taxi_zones/taxi_zones.shp")
shapefile

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


According to the shapefile "taxi_zones.shp" above, the highest objectID, which is the pickup location ID and drop off location ID, is 263. Therefore, in the Cleaning & filtering step, we remove the IDs that are greater than 263.

In [27]:
nc_counties = shapefile['geometry']
nc_counties = nc_counties.to_crs(4326)
nc_counties['lon'] = nc_counties.centroid.x
#nc_counties['lat'] = nc_counties.centroid.y

/var/folders/sl/ygvmf08n6s5cz73c1jv6rbnw0000gn/T/ipykernel_26756/3792372224.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nc_counties['lon'] = nc_counties.centroid.x
